In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('Datasets/train.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [151]:
df.depth.value_counts()

61.9    10781
62.0    10150
61.8     9270
62.1     8866
61.6     8534
        ...  
69.4        1
68.1        1
52.1        1
55.8        1
55.0        1
Name: depth, Length: 153, dtype: int64

In [74]:
# Split dataset and Drop Columns:

X = df.drop(columns=['id','price'])

y= df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [152]:
from feature_engine.encoding import RareLabelEncoder

rare_encoder = RareLabelEncoder()

#rare_encoder.fit(X_train)
#X_train = rare_encoder.transform(X_train)

In [153]:
#from feature_engine.encoding import OneHotEncoder

#hot_encoder = OneHotEncoder()

In [154]:
from feature_engine.encoding import OrdinalEncoder

cat_encoder = OrdinalEncoder(encoding_method='ordered')

#cat_encoder.fit(X_train)
#X_train = cat_encoder.transform(X_train)

In [155]:
from feature_engine.outliers import Winsorizer

capper = Winsorizer(capping_method="gaussian", tail="both")

#capper.fit(X_train)
#X_train = capper.transform(X_train)

In [156]:
from feature_engine.transformation import YeoJohnsonTransformer

yeo_trans = YeoJohnsonTransformer()

#yeo_trans.fit(X_train)
#X_train = yeo_trans.transform(X_train)

In [157]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#scaler.fit(X_train)
#X_train = scaler.transform(X_train)

In [158]:
from feature_engine.selection import DropConstantFeatures, SmartCorrelatedSelection, DropDuplicateFeatures

# Drop constant
drop_const = DropConstantFeatures(tol=0.90)

#drop_const.fit(X_train)
#X_train = drop_const.transform(X_train)

# Drop duplicates
drop_dup = DropDuplicateFeatures()

#drop_dup.fit(X_train)
#X_train = drop_dup.transform(X_train)

# Drop correlated
drop_corr = SmartCorrelatedSelection(method='spearman',
                                     selection_method='variance',
                                     estimator=None, scoring='roc_auc')

#drop_corr.fit(X_train, y_train)
#X_train = drop_corr.transform(X_train)

In [160]:
# Import make_pipeline
from sklearn.pipeline import make_pipeline

# Define pipeline steps

transformer_pipe = make_pipeline(rare_encoder,
                                 #hot_encoder,
                                 cat_encoder,
                                 capper,
                                 yeo_trans,
                                 scaler,
                                # drop_const,
                                # drop_dup,
                                # drop_corr
                                 )

#fit and transform
#X_train = transformer_pipe.fit_transform(X_train,y_train)
#X_test = transformer_pipe.transform(X_test)

In [88]:

# define your object using default hyperameters

catb_reg = CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False)

# fit your training data

catb_reg.fit(X_train, y_train)
catb_reg.score(X_test, y_test)

0.9805024819294389

In [89]:
catb_reg.predict(X_test)

rsme_test = mean_squared_error(y_test, catb_reg.predict(X_test), squared=False)
rsme_test

568.0529612157402

# Stacking:

In [12]:
# Import algorithms to try
import time
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor

In [ ]:
XGBRegressor(alpha= 0.04996268773333479, colsample_bytree= 0.8261293830116522, 
             gamma= 2, learning_rate= 0.0774130167276674, 
             max_depth= 6, min_child_weight= 2, n_estimators= 308, 
             reg_lambda= 0.24843411844875563, subsample= 0.735787253770912, random_state=42)

In [177]:
# 85-15

base_models = []

base_models.append(('lr', LinearRegression()))
base_models.append(('ctb', CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False, random_state=42))) 
base_models.append(('rf', RandomForestRegressor(random_state=42)))

meta_model = LinearRegression()

reg_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

reg_model.fit(X_train, y_train)
reg_model.score(X_test, y_test)

0.9796362957901991

In [178]:
stack_reg = reg_model.predict(X_test)

rsme_test = mean_squared_error(y_test, stack_reg, squared=False)
rsme_test

575.7521515761919

In [25]:
# 85-15 + SGD + CatBoost + RandomForest

base_models = []

base_models.append(('lr', SGDRegressor()))
base_models.append(('ctb', CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False, random_state=42))) 
base_models.append(('rf', RandomForestRegressor(random_state=42)))

meta_model = SGDRegressor()

reg_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

reg_model.fit(X_train, y_train)
reg_model.score(X_test, y_test)

-1.8371359082692117e+23

In [ ]:
stack_reg = reg_model.predict(X_test)

rsme_test = mean_squared_error(y_test, stack_reg, squared=False)
rsme_test

In [25]:
# 85-15 + LinReg + CTBoost + KNN Reg (no duplicate, corr ...)

base_models = []

base_models.append(('lr', LinearRegression()))
base_models.append(('ctb', CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False, random_state=42))) 
base_models.append(('rf', KNeighborsRegressor(algorithm= 'auto', leaf_size= 50,n_neighbors= 10, weights= 'distance')))

meta_model = LinearRegression()

reg_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

reg_model.fit(X_train, y_train)
reg_model.score(X_test, y_test)

0.9787635873460732

In [26]:
stack_reg = reg_model.predict(X_test)

rsme_test = mean_squared_error(y_test, stack_reg, squared=False)
rsme_test

587.9599677196173

In [13]:
# Create table with models and its performance
# IMPORTANT NOTE: change metrics and algorithms as you see fit

# Define algorithms to try
algorithms = []
algorithms.append(('ElasticNet', ElasticNet()))
algorithms.append(('Linear Regressio', LinearRegression()))
algorithms.append(('Random Forest', RandomForestRegressor()))
algorithms.append(('XG Boost', XGBRegressor()))
algorithms.append(('KNN Regressor', KNeighborsRegressor()))
algorithms.append(('SGD Regressor', SGDRegressor()))
#algorithms.append(('Genetic', TPOTRegressor()))
algorithms.append(('CatBoost', CatBoostRegressor(verbose=False)))

# Initialize lists
names = []
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []
cross_validation_scores = []
runtime = []

# Iterate ofer each element in algorithms
for name, reg in algorithms:
  t1 = time.time()
  names.append(name)
  reg.fit(X_train, y_train)
  train_rmse.append(mean_squared_error(y_train, reg.predict(X_train), squared=False))
  test_rmse.append(mean_squared_error(y_test, reg.predict(X_test), squared=False))
  train_r2.append(r2_score(y_train, reg.predict(X_train)))
  test_r2.append(r2_score(y_test, reg.predict(X_test)))
  cross_validation_scores.append(cross_val_score(reg, np.vstack((X_train, X_test)), np.hstack((y_train, y_test))).mean())
  t2 = time.time()
  runtime.append(t2-t1)

# Create comparison dataframe
model_comparison = pd.DataFrame({'Algorithm': names,
                                 'Train RMSE': train_rmse,
                                 'Test_RMSE': test_rmse,
                                 'Train r2 score': train_r2,
                                 'Test r2 score': test_r2,
                                 'Cross validation score': cross_validation_scores,
                                 'Runtime': runtime,})

# Sort by Test_RMSE
model_comparison.sort_values(by=['Test_RMSE'])

,Algorithm,Train RMSE,Test_RMSE,Train r2 score,Test r2 score,Cross validation score,Runtime
6,CatBoost,529.035073,577.017662,0.982804,0.979547,0.979526,34.910010
3,XG Boost,512.063672,590.214617,0.983889,0.978600,0.978592,8.428874
2,Random Forest,229.621823,617.286981,0.996760,0.976592,0.976843,290.280815
4,KNN Regressor,580.368681,705.203198,0.979305,0.969450,0.968648,23.702269
5,SGD Regressor,1725.127826,1745.821453,0.817146,0.812766,0.816244,7.759772
1,Linear Regressio,1723.171659,1748.042547,0.817561,0.812289,0.816510,0.189647
0,ElasticNet,2005.016165,2007.023545,0.753000,0.752548,0.752971,0.207061


# Train - Validation - Test:

In [41]:
# Split dataset and Drop Columns:

X = df.drop(columns=['id','price'])

y= df['price']

X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.20, random_state = 42)

In [42]:

X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.50, random_state = 42)

In [43]:
#fit and transform
X_train = transformer_pipe.fit_transform(X_train,y_train)
X_val = transformer_pipe.transform(X_val)
X_test = transformer_pipe.transform(X_test)

In [172]:
# Catboost Parameter Tuning:

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint

distributions = {#'n_estimators': randint(50, 500),
                 'depth': randint(4, 9),
                 'learning_rate': uniform(0.001,0.01),
                 'l2_leaf_reg': randint(2,10),
                 'random_strength': uniform(0, 10),
                 }

cat_obj_rs = CatBoostRegressor(verbose=False,random_state=42)
cat_clf_rs = RandomizedSearchCV(cat_obj_rs, distributions, n_iter=250, verbose=False)

search = cat_clf_rs.fit(X_val, y_val)

print(cat_clf_rs.best_params_)
print(cat_clf_rs.best_score_)

{'depth': 8, 'l2_leaf_reg': 3, 'learning_rate': 0.01090977882887642, 'random_strength': 1.603782684502414}
0.9789516777249258


In [44]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint

In [45]:
param_grid = {  
     'bootstrap': [True, False],
     'max_depth': [2,6,8,10,20],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestRegressor(),
                           param_grid=param_grid)
grid_search.fit(X_val, y_val)
print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=10)
{'bootstrap': True, 'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 10}
0.976010579283342


In [22]:
param_grid = {  
    'alpha': [0.0001, 0.0005, 0.001, 0.005, 0.01], # learning rate
    'penalty': ['l2', 'l1', 'elasticnet'],
    #'n_iter_no_change': [5, 10, 50, 100, 500, 1000, 1500], # number of epochs
    }

grid_search_sgd = GridSearchCV(SGDRegressor(),
                           param_grid=param_grid)
grid_search_sgd.fit(X_val, y_val)
print(grid_search_sgd.best_estimator_)
print(grid_search_sgd.best_params_)
print(grid_search_sgd.best_score_)

SGDRegressor(alpha=0.01, penalty='l1')
{'alpha': 0.01, 'penalty': 'l1'}
0.8188303454132113


In [34]:
param_grid = {  
    'n_neighbors': [2,3,4,5,6,7,8,9,10], # learning rate
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10,20,30,50,100,150]
    }

grid_search_knn = GridSearchCV(KNeighborsRegressor(),
                           param_grid=param_grid)
grid_search_knn.fit(X_val, y_val)
print(grid_search_knn.best_estimator_)
print(grid_search_knn.best_params_)
print(grid_search_knn.best_score_)

KNeighborsRegressor(leaf_size=50, n_neighbors=10, weights='distance')
{'algorithm': 'auto', 'leaf_size': 50, 'n_neighbors': 10, 'weights': 'distance'}
0.9633153508102579


In [193]:
# Random Forest Parameter Tuning:

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint

distributions = {'max_depth': randint(4, 9),
                 'learning_rate': uniform(0.001,0.01),
                 'min_samples_split': randint(2, 10),
                 'min_samples_leaf': randint(2, 10),
                 }

rf_obj_rs = LinearRegression()(verbose=False,random_state=42)
rf_clf_rs = RandomizedSearchCV(cat_obj_rs, distributions, n_iter=50, verbose=False)

search = rf_clf_rs.fit(X_val, y_val)

print(rf_clf_rs.best_params_)
print(rf_clf_rs.best_score_)

#max_leaf_nodes, max_depth and min_samples_leaf

TypeError: __init__() got an unexpected keyword argument 'min_samples_leaf'

In [191]:
!pip install scipy

# Best Model #1;

Split 70-30

In [73]:
# define your object using default hyperameters

catb_reg = CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False)

# fit your training data

catb_reg.fit(X_train, y_train)
catb_reg.score(X_test, y_test)

0.9796175745301323

In [154]:
# define your object using default hyperameters

catb_reg = CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False, random_state=42)

# fit your training data

catb_reg.fit(X_train, y_train)
catb_reg.score(X_test, y_test)

0.9792494590249877

In [151]:
catb_reg = catb_reg.predict(X_test)

rsme_test = mean_squared_error(y_test, catb_reg, squared=False)
rsme_test

579.0562968058885

# Best Model - #2:

In [13]:
# 85-15 + LinReg + CTBoost + KNN Reg

base_models = []

base_models.append(('lr', LinearRegression()))
base_models.append(('ctb', CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False, random_state=42))) 
base_models.append(('rf', KNeighborsRegressor(algorithm= 'auto', leaf_size= 50,n_neighbors= 10, weights= 'distance')))

meta_model = LinearRegression()

reg_model_stack = StackingRegressor(estimators=base_models, final_estimator=meta_model)

reg_model_stack.fit(X_train, y_train)
reg_model_stack.score(X_test, y_test)

0.9796080722804885

# Best Model #3

In [101]:
# 85-15

base_models = []

base_models.append(('lr', LinearRegression()))
base_models.append(('ctb', CatBoostRegressor(depth= 11, l2_leaf_reg= 3, learning_rate= 0.01037204787019528, random_strength= 0.37631316886909794, verbose=False, random_state=42))) 
base_models.append(('rf', RandomForestRegressor()))

meta_model = LinearRegression()

reg_model_stack2 = StackingRegressor(estimators=base_models, final_estimator=meta_model)

reg_model_stack2.fit(X_train, y_train)
reg_model_stack2.score(X_test, y_test)

0.9805724432216615

In [102]:
reg_model_stack2.predict(X_test)

rsme_test = mean_squared_error(y_test, reg_model_stack2.predict(X_test), squared=False)
rsme_test

567.0328971840235

# Best Model #4

In [99]:
# 85-15

base_models = []

base_models.append(('lr', LinearRegression()))
base_models.append(('ctb', CatBoostRegressor(n_estimators=10000, verbose=False))) 
base_models.append(('rf', RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=10)))

meta_model = LinearRegression()

reg_model_stack2 = StackingRegressor(estimators=base_models, final_estimator=meta_model)

reg_model_stack2.fit(X_train, y_train)
reg_model_stack2.score(X_test, y_test)

0.9803953972306763

In [100]:
reg_model_stack2.predict(X_test)

rsme_test = mean_squared_error(y_test, reg_model_stack2.predict(X_test), squared=False)
rsme_test

569.610761699019

In [ ]:
max_depth=20, min_samples_leaf=2, min_samples_split=10

# Model 5

In [72]:
from catboost import Pool, cv

cv_data = X_train

labels = y_train


cv_dataset = Pool(data=cv_data,
                  label=labels,
                  )

params = {'depth': 11, 
          'l2_leaf_reg': 3, 
          'learning_rate': 0.01037204787019528, 
          'random_strength': 0.37631316886909794,
          "loss_function": 'RMSE',
          "verbose": False}

scores = cv(cv_dataset,
            params,
            fold_count=2,
           return_models=True,
           verbose=False)


Training on fold [0/2]

bestTest = 587.0647283
bestIteration = 999

Training on fold [1/2]

bestTest = 570.5150454
bestIteration = 995



NameError: name 'cv_results' is not defined

# Final Predictions for test.csv:

In [90]:
df_test = pd.read_csv('Datasets/test.csv')
X_to_test = df_test.drop(columns='id')
X_to_test.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.35,Ideal,D,VS2,62.3,56.0,4.51,4.54,2.82
1,0.77,Very Good,F,SI2,62.8,56.0,5.83,5.87,3.68
2,0.71,Ideal,I,VS2,61.9,53.0,5.77,5.74,3.55
3,0.33,Ideal,G,VVS2,61.6,55.0,4.44,4.42,2.73
4,1.20,Very Good,I,VS2,62.7,56.0,6.75,6.79,4.24


In [91]:
#Data transformations:
X_to_test = transformer_pipe.transform(X_to_test)

In [92]:
# Predictions using my model:

y_pred = catb_reg.predict(X_to_test)

In [93]:
y_pred

array([ 888.50380084, 2599.57886941, 2363.89488675, ..., 7751.24096594,
       6759.70391039, 4219.10733882])

In [94]:
output_price = pd.DataFrame(y_pred, columns=['price']) #elbow curve 

In [95]:
output_price

,price
0,888.503801
1,2599.578869
2,2363.894887
3,842.430688
4,5886.924016
...,...
129045,3898.460743
129046,2570.210590
129047,7751.240966
129048,6759.703910


In [96]:
output_price['price'] = pd.DataFrame(y_pred) #elbow curve 

df_out = pd.concat([df_test.reset_index(drop='Tru‌e'),output_price['price'].reset_index(drop='Tru‌e')],axis=1)
df_out

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,193573,0.35,Ideal,D,VS2,62.3,56.0,4.51,4.54,2.82,888.503801
1,193574,0.77,Very Good,F,SI2,62.8,56.0,5.83,5.87,3.68,2599.578869
2,193575,0.71,Ideal,I,VS2,61.9,53.0,5.77,5.74,3.55,2363.894887
3,193576,0.33,Ideal,G,VVS2,61.6,55.0,4.44,4.42,2.73,842.430688
4,193577,1.20,Very Good,I,VS2,62.7,56.0,6.75,6.79,4.24,5886.924016
...,...,...,...,...,...,...,...,...,...,...,...
129045,322618,0.72,Ideal,D,VVS2,62.0,56.0,5.75,5.78,3.57,3898.460743
129046,322619,0.70,Premium,D,SI1,59.6,62.0,5.77,5.74,3.43,2570.210590
129047,322620,1.01,Premium,G,VVS2,62.3,58.0,6.44,6.41,4.01,7751.240966
129048,322621,1.35,Ideal,D,I1,62.0,56.0,7.05,7.08,4.38,6759.703910


In [97]:
submit_6 = df_out[['id','price']]
submit_6
                

,id,price
0,193573,888.503801
1,193574,2599.578869
2,193575,2363.894887
3,193576,842.430688
4,193577,5886.924016
...,...,...
129045,322618,3898.460743
129046,322619,2570.210590
129047,322620,7751.240966
129048,322621,6759.703910


In [98]:
submit_6.to_csv('submit_6.csv',index=False)

https://www.kaggle.com/code/satoshiss/gemstone-price-prediction-s3e8

In [103]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [110]:
df_selected = df.loc[0:9,['id','price','cut']]
df_selected

,id,price,cut
0,0,13619,Premium
1,1,13387,Very Good
2,2,2772,Ideal
3,3,666,Ideal
4,4,14453,Premium
5,5,7506,Very Good
6,6,3229,Ideal
7,7,6224,Premium
8,8,886,Ideal
9,9,421,Good


In [119]:
from sklearn.model_selection import KFold

X = df.drop(columns=['price'])
y = df['price']



5

In [120]:
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [125]:
for i, (train_index, test_index) in enumerate(kf.split(X)):
     print(f"Fold {i}:")
     print(f"  Train: index={train_index}")
     print(f"  Test:  index={test_index}")
    
        
    

Fold 0:
  Train: index=[2 3 4 5 6 7 8 9]
  Test:  index=[0 1]
Fold 1:
  Train: index=[0 1 4 5 6 7 8 9]
  Test:  index=[2 3]
Fold 2:
  Train: index=[0 1 2 3 6 7 8 9]
  Test:  index=[4 5]
Fold 3:
  Train: index=[0 1 2 3 4 5 8 9]
  Test:  index=[6 7]
Fold 4:
  Train: index=[0 1 2 3 4 5 6 7]
  Test:  index=[8 9]


# Kaggle

https://www.kaggle.com/code/satoshiss/gemstone-price-prediction-s3e8

In [143]:
N=5

train_list = []
test_list = []

# stratifiedKfold by department
kfold = KFold(n_splits=N, random_state=10,shuffle=True)
X = df.drop(columns=['price'])
y = df['price'] 

for i, (train_index,test_index) in enumerate(kfold.split(X)):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    train_list.append((X_train,y_train))
    test_list.append((X_test,y_test))
    
    #print(f"Fold {i}:")
    #print(train_list[i][0])
    #print(test_list[i][0])
    #print(train_list[i][1])
    #print(test_list[i][1])

In [171]:
N=10

train_list = []
test_list = []

X_train_fold = []
X_test_fold = []
y_train_fold = []
y_test_fold = []


# stratifiedKfold by department
kfold = KFold(n_splits=N, random_state=10,shuffle=True)
X = df.drop(columns=['price'])
y = df['price'] 

for i, (train_index,test_index) in enumerate(kfold.split(X)):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    X_train_fold.append((X_train))
    X_test_fold.append((X_test))
    y_train_fold.append((y_train))
    y_test_fold.append((y_test))
    
   # print(f"Fold {i}:")
   # print(X_train_fold[i])
   # print(X_test_fold[i])
   # print(y_train_fold[i])
   # print(y_test_fold[i])

In [175]:
predictions=[]
rmses=[]


for  i in range(N):
    model = CatBoostRegressor(n_estimators=10000, verbose=False)

    
    y_train = y_train_fold[i]
    X_train = X_train_fold[i]
    
    y_test = y_test_fold[i]
    X_test = X_test_fold[i]
    
    X_train = transformer_pipe.fit_transform(X_train,y_train)
    X_test = transformer_pipe.transform(X_test)
   
    # fit the model
    model.fit(X_train,y_train,
             early_stopping_rounds=100, 
             eval_set=[(X_test, y_test)],
             )
    
    # evaluate on test data
    pre = model.predict(X_test)
    
    rmse = mean_squared_error(y_test,pre,squared=False)
    rmses.append(rmse)
    
    print(f'Fold {i+1}')
    print('Accuracy on validation data: {:.1f}'.format(rmse))
    
    print()
    # two votes if scored well 
    #pre = model.predict(test_df)
    #predictions.append(pre)
print()
print(f'Average RMSE: {np.mean(rmses)}')

Fold 1
Accuracy on validation data: 586.9

Fold 2
Accuracy on validation data: 586.2

Fold 3
Accuracy on validation data: 564.5

Fold 4
Accuracy on validation data: 588.0

Fold 5
Accuracy on validation data: 575.5

Fold 6
Accuracy on validation data: 569.7

Fold 7
Accuracy on validation data: 562.8

Fold 8
Accuracy on validation data: 562.5

Fold 9
Accuracy on validation data: 566.8

Fold 10
Accuracy on validation data: 580.1


Average RMSE: 574.3077075177893


In [ ]:
predictions= [np.mean(a) for a in zip(*predictions)]

In [ ]:
true_value = y_test
predicted_value = y_pred_test

plt.figure(figsize=(5,5))
plt.scatter(true_value, predicted_value, c='crimson')

p1 = max(max(predicted_value), max(true_value))
p2 = min(min(predicted_value), min(true_value))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
reg_model_stack2.predict(X_test)

rsme_test = mean_squared_error(y_test, reg_model_stack2.predict(X_test), squared=False)
rsme_test

In [134]:
train_list[3][0]

,id,cut
0,0,Premium
1,1,Very Good
2,2,Ideal
3,3,Ideal
4,4,Premium
5,5,Very Good
7,7,Premium
8,8,Ideal
9,9,Good


In [122]:
X.loc[[0,1]]

,id,cut
0,0,Premium
1,1,Very Good


In [123]:
y.loc[[0,1]]

0    13619
1    13387
Name: price, dtype: int64